### CAPSTONE PROJECT: "WHERE TO MOVE IN HELSINKI?"

In [4]:
#Import all the necessary packages

import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

import requests 
import ssl
import certifi

from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

import geocoder
from geopy.geocoders import Nominatim

#conda install -c conda-forge pyproj

#from pyproj import Proj, transform

print('Libraries imported.')


Libraries imported.


In [27]:
from pathlib import Path
data_folder= Path("C:/Users/maria.tornroos/Desktop/")
file_to_open=data_folder / "Helsinki_osoiteluettelo.csv"


#This file has addresses, postal codes and coordinates

helsinki=pd.read_csv(file_to_open)

In [28]:
helsinki.head()

,FID,id,katunimi,gatan,osoitenumero,osoitenumero2,osoitekirjain,n,e,kaupunki,staden,tyyppi,postinumero,postitoimipaikka,osoitenumero_teksti,geom,datanomistaja,paivitetty_tietopalveluun
0,Helsinki_osoiteluettelo.0,0,Aamuruskontie,Morgonrådevägen,22.0,NaN,b,6685815,25501108,Helsinki,Helsingfors,1,750.0,Helsinki,22b,POINT (6685815 25501108),NaN,2020-01-30
1,Helsinki_osoiteluettelo.1,1,Aamuruskontie,Morgonrådevägen,24.0,NaN,NaN,6685809,25501073,Helsinki,Helsingfors,1,750.0,Helsinki,24,POINT (6685809 25501073),NaN,2020-01-30
2,Helsinki_osoiteluettelo.2,2,Aamuruskontie,Morgonrådevägen,25.0,NaN,b,6685871,25501088,Helsinki,Helsingfors,1,750.0,Helsinki,25b,POINT (6685871 25501088),NaN,2020-01-30
3,Helsinki_osoiteluettelo.3,3,Aamuruskontie,Morgonrådevägen,26.0,NaN,NaN,6685820,25501051,Helsinki,Helsingfors,1,750.0,Helsinki,26,POINT (6685820 25501051),NaN,2020-01-30
4,Helsinki_osoiteluettelo.4,4,Aamuruskontie,Morgonrådevägen,27.0,NaN,b,6685896,25501082,Helsinki,Helsingfors,1,750.0,Helsinki,27b,POINT (6685896 25501082),NaN,2020-01-30


In [29]:

#Dropping missing in postal code
helsinki.dropna(subset=['postinumero'],axis=0)

helsinki['postinumero'] = pd.to_numeric(helsinki['postinumero'], errors='coerce').fillna(0).astype(int)
helsinki['postinumero'].dtype
#Filling postalcode with leading zeros
helsinki['postinumero'] = helsinki['postinumero'].apply(lambda x: '{0:0>5}'.format(x))
helsinki.drop(columns=['FID','id','gatan','osoitenumero2','osoitekirjain','kaupunki','staden','tyyppi','osoitenumero_teksti','datanomistaja','paivitetty_tietopalveluun'],inplace=True)
helsinki.rename(columns={'e':'longitude','n':'latitude'}, inplace=True)
helsinki.head()

,katunimi,osoitenumero,latitude,longitude,postinumero,postitoimipaikka,geom
0,Aamuruskontie,22.0,6685815,25501108,00750,Helsinki,POINT (6685815 25501108)
1,Aamuruskontie,24.0,6685809,25501073,00750,Helsinki,POINT (6685809 25501073)
2,Aamuruskontie,25.0,6685871,25501088,00750,Helsinki,POINT (6685871 25501088)
3,Aamuruskontie,26.0,6685820,25501051,00750,Helsinki,POINT (6685820 25501051)
4,Aamuruskontie,27.0,6685896,25501082,00750,Helsinki,POINT (6685896 25501082)


In [30]:
#This file has neighbourhood names
file_to_open2=data_folder / "PKS_Postinumeroalueet_2019.xlsx"

helsinki_postialue=pd.read_excel(file_to_open2)
helsinki_postialue['Postinnumero'] = helsinki_postialue['Postinnumero'].apply(lambda x: '{0:0>5}'.format(x))
helsinki_postialue.drop(columns=['Postitoimipaikka','Toimipaikka_ru','Nimi_Ru','Kunta','Kunta_nro'],inplace=True)
helsinki_postialue.rename(columns={'Postinnumero':'postinumero'},inplace=True)
helsinki_postialue.head()

,postinumero,Nimi
0,00100,Helsinki Keskusta - Etu-Töölö
1,00120,Punavuori
2,00130,Kaartinkaupunki
3,00140,Kaivopuisto - Ullanlinna
4,00150,Eira - Hernesaari


Merge the two datasets

In [31]:
helsinki_osoitteet=pd.merge(helsinki,helsinki_postialue,how='left',on='postinumero')

In [32]:
helsinki_osoitteet.tail()


,katunimi,osoitenumero,latitude,longitude,postinumero,postitoimipaikka,geom,Nimi
53527,Meilahdensilta,NaN,6675547,25493527,00000,NaN,POINT (6675547 25493527),NaN
53528,Suomensuonpuisto,NaN,6672298,25502377,00000,NaN,POINT (6672298 25502377),NaN
53529,Tuohiaukio (tori tai aukio),NaN,6682104,25492124,00000,NaN,POINT (6682104 25492124),NaN
53530,Toiskantori (tori tai aukio),NaN,6681697,25491839,00000,NaN,POINT (6681697 25491839),NaN
53531,Panosaukio (tori tai aukio),NaN,6680048,25503488,00000,NaN,POINT (6680048 25503488),NaN


Get rid of the NaNs

In [33]:
helsinki_osoitteet.dropna(subset=['Nimi'],axis=0, inplace=True)

Get the correct coordinates

In [34]:
inProj, outProj = Proj(init='epsg:3879'), Proj(init='epsg:4326')
helsinki_osoitteet['newLon'], helsinki_osoitteet['newLat'] = transform(inProj, outProj, helsinki_osoitteet['longitude'].tolist(), helsinki_osoitteet['latitude'].tolist())

In [35]:
helsinki_osoitteet.head()

,katunimi,osoitenumero,latitude,longitude,postinumero,postitoimipaikka,geom,Nimi,newLon,newLat
0,Aamuruskontie,22.0,6685815,25501108,00750,Helsinki,POINT (6685815 25501108),Puistola,25.020029,60.284900
1,Aamuruskontie,24.0,6685809,25501073,00750,Helsinki,POINT (6685809 25501073),Puistola,25.019396,60.284846
2,Aamuruskontie,25.0,6685871,25501088,00750,Helsinki,POINT (6685871 25501088),Puistola,25.019668,60.285402
3,Aamuruskontie,26.0,6685820,25501051,00750,Helsinki,POINT (6685820 25501051),Puistola,25.018999,60.284945
4,Aamuruskontie,27.0,6685896,25501082,00750,Helsinki,POINT (6685896 25501082),Puistola,25.019560,60.285627


Clean the dataset

In [36]:
helsinki_df=helsinki_osoitteet.copy()
helsinki_df.drop_duplicates(subset=["Nimi"], keep="last", inplace=True)
helsinki_df.tail()
helsinki_df.shape
helsinki_df.reset_index(inplace=True)
helsinki_df.drop(columns=['index'],inplace=True)
helsinki_df.drop(columns=['Unnamed: 0','latitude','longitude','geom','postinumero','postitoimipaikka'],inplace=True)
helsinki_df.drop(columns=['katunimi','osoitenumero'],inplace=True)
helsinki_df.rename(columns={'Nimi':'Area'},inplace=True)

,katunimi,osoitenumero,latitude,longitude,postinumero,postitoimipaikka,geom,Nimi,newLon,newLat
0,Talin siirtolapuutarha (siirtolapuutarha tai m...,102.0,6678443,25492367,00360,Helsinki,POINT (6678443 25492367),Pajamäki,24.862298,60.218664
1,Kivihaantie,1.0,6677415,25494663,00310,Helsinki,POINT (6677415 25494663),Kivihaka,24.903746,60.209473
2,Topeliuksenkatu,30.0,6675225,25495027,00290,Helsinki,POINT (6675225 25495027),Meilahden sairaala-alue,24.910364,60.189822
3,Amiraali Cronstedtin ranta,17.0,6675203,25501361,00810,Helsinki,POINT (6675203 25501361),Herttoniemi,25.024531,60.189653
4,Tenholantie,1.0,6676667,25494547,00280,Helsinki,POINT (6676667 25494547),Ruskeasuo,24.901674,60.202758
5,Mäntytie,16.0,6675711,25494225,00270,Helsinki,POINT (6675711 25494225),Pohjois-Meilahti,24.895895,60.194173
6,Suomenlinna C,54.0,6670442,25499330,00190,Helsinki,POINT (6670442 25499330),Suomenlinna,24.987939,60.146922
7,Pakettikatu,1.0,6678272,25495768,00230,Helsinki,POINT (6678272 25495768),Ilmala,24.923657,60.217178
8,Hevostallintie,7.0,6670942,25502707,00860,Helsinki,POINT (6670942 25502707),Santahamina,25.048735,60.151402
9,Haarniskatie,9.0,6678218,25505681,00910,Helsinki,POINT (6678218 25505681),Puotila,25.102481,60.216676


In [10]:
helsinki_df.head()

,Area,newLon,newLat
0,Pajamäki,24.862298,60.218664
1,Kivihaka,24.903746,60.209473
2,Meilahden sairaala-alue,24.910364,60.189822
3,Herttoniemi,25.024531,60.189653
4,Ruskeasuo,24.901674,60.202758


Check the coordinates of Helsinki

In [11]:
address = 'Helsinki'

geolocator = Nominatim(user_agent="hel_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Helsinki are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Helsinki are 60.1674098, 24.9425769.


Check the venues in a random neighbourhood

In [12]:
helsinki_df.loc[33, 'Area']

'Taka-Töölö'

In [13]:
neighbourhood_latitude = helsinki_df.loc[33, 'newLat'] # neighbourhood latitude value
neighbourhood_longitude = helsinki_df.loc[33, 'newLon'] # neighbourhood longitude value

neighbourhood_name = helsinki_df.loc[33, 'Area'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Taka-Töölö are 60.18327692809014, 24.9214287941142.


In [15]:

LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=QU5LW5XUZ30OWUFDP1SLWKXYKLACNJTISCR4AIKGEUTEU2VM&client_secret=OOK5SOR0JBBNQHCCG3AAXYI3ASBMIBRNBN4Z4KD0ENN2WIWX&v=20180605&ll=60.18327692809014,24.9214287941142&radius=500&limit=100'

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e40fd8447e0d6001b2f5e9d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Taka-Töölö',
  'headerFullLocation': 'Taka-Töölö, Helsinki',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': 60.18777693259014,
    'lng': 24.930462093829657},
   'sw': {'lat': 60.178776923590135, 'lng': 24.91239549439874}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '596b74b66f706a53b11b1236',
       'name': 'Daddy Greens Pizzabar',
       'location': {'address': 'Humalistonkatu 3',
        'lat': 60.18498218861514,
        'lng': 24.920734371474094,
        'labeledLatLngs': [{'label': 'display',
          'l

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Daddy Greens Pizzabar,Pizza Place,60.184982,24.920734
1,Korjaamo Summer Garden,Beer Garden,60.184098,24.920041
2,Korjaamo Culture Factory,Event Space,60.184097,24.919950
3,Korjaamo Kino,Indie Movie Theater,60.184262,24.919897
4,Linko,Pizza Place,60.181740,24.925074


Loop to get all the venues for all neighbourhoods

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Address', 
                  'Address Latitude', 
                  'Address Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
helsinki_venues = getNearbyVenues(names=helsinki_df['Area'],
                                   latitudes=helsinki_df['newLat'],
                                   longitudes=helsinki_df['newLon']
                                  )

Pajamäki
Kivihaka
Meilahden sairaala-alue
Herttoniemi
Ruskeasuo
Pohjois-Meilahti
Suomenlinna
Ilmala
Santahamina
Puotila
Pikku Huopalahti
Koskela-Helsinki
Lassila
Pohjois-Haaga
Kaivopuisto - Ullanlinna
Kalasatama
Pohjois-Vuosaari
Länsi-Pasila
Kulosaari
Vattuniemi
Vallila
Itä-Pasila
Kallio
Toukola-Vanhakaupunki
Sörnäinen
Roihuvuori
Pitäjänmäen teollisuusalue
Siltamäki
Viikki
Veräjämäki
Jätkäsaari
Eira - Hernesaari
Kannelmäki
Taka-Töölö
Munkkivuori-Niemenmäki
Kuusisaari-Lehtisaari
Kamppi - Ruoholahti
Roihupellon teollisuusalue
Laajasalo
Pihlajamäki
Kruununhaka
Myllypuro
Maununneva
Kaitalahti
Munkkiniemi
Malminkartano
Tapaninvainio
Mellunmäki
Tuomarinkylä-Torppar
Punavuori
Konala
Itäkeskus-Marjaniemi
Helsinki Keskusta - Etu-Töölö
Kontula - Vesala
Itäsalmi
Jollas
Paloheinä
Itä-Pakila
Länsi-Pakila
Aurinkolahti
Puotinharju
Tammisalo
Kaartinkaupunki
Puistola
Länsi-Herttoniemi
Pukinmäki-Savela
Metsälä-Etelä-Oulunkylä
Verkkosaari
Oulunkylä-Patola
Lauttasaari
Etelä-Haaga
Vartioharju
Etelä-Laajasa

In [16]:
print(helsinki_venues.shape)
helsinki_venues.head()

(1556, 7)


,Address,Address Latitude,Address Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Pajamäki,60.218664,24.862298,Strömbergin puisto,60.220224,24.865653,Park
1,Pajamäki,60.218664,24.862298,Fitness24Seven,60.217097,24.869112,Gym / Fitness Center
2,Pajamäki,60.218664,24.862298,Talin siirtolapuutarha,60.217073,24.862052,Garden
3,Pajamäki,60.218664,24.862298,Trendin Sali,60.217110,24.869564,Gym
4,Kivihaka,60.209473,24.903746,Arena Center,60.206271,24.901650,Recreation Center


In [17]:
print('There are {} uniques categories.'.format(len(helsinki_venues['Venue Category'].unique())))

There are 240 uniques categories.


Onehot encoding

In [21]:
helsinki_onehot = pd.get_dummies(helsinki_venues[['Venue Category']], prefix="", prefix_sep="")

helsinki_onehot['Area'] = helsinki_venues['Address'] 

fixed_columns = [helsinki_onehot.columns[-1]] + list(helsinki_onehot.columns[:-1])
helsinki_onehot = helsinki_onehot[fixed_columns]

helsinki_onehot.head()

,Area,ATM,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Campground,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Science Building,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Forest,Fraternity House,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Rink,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Library,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mountain,Multiplex,Museum,Music School,Music Store,Music Venue,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Port,Post Office,Pub,Record Shop,Recreation Center,Recycling Facility,Rental Car Location,Rental Service,Reservoir,Resort,Restaurant,River,Road,Russian Restaurant,Salad Place,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Repair,Shopping Mall,Skate Park,Skating Rink,Ski Trail,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,Pajamäki,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Pajamäki,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [22]:
helsinki_grouped = helsinki_onehot.groupby('Area').mean().reset_index()
helsinki_grouped

,Area,ATM,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Campground,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Science Building,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Forest,Fraternity House,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Rink,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Library,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mountain,Multiplex,Museum,Music School,Music Store,Music Venue,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Port,Post Office,Pub,Record Shop,Recreation Center,Recycling Facility,Rental Car Location,Rental Service,Reservoir,Resort,Restaurant,River,Road,Russian Restaurant,Salad Place,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Repair,Shopping Mall,Skate Park,Skating Rink,Ski Trail,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,Aurinkolahti,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

Picking 5 top venues in each neighbourhood based on frequency

In [23]:
num_top_venues = 5

for hood in helsinki_grouped['Area']:
    print("----"+hood+"----")
    temp = helsinki_grouped[helsinki_grouped['Area'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aurinkolahti----
           venue  freq
0          Beach  0.14
1  Grocery Store  0.14
2     Playground  0.14
3         Resort  0.07
4     Restaurant  0.07


----Eira - Hernesaari----
                     venue  freq
0                     Park  0.10
1  Scandinavian Restaurant  0.07
2                   Bakery  0.05
3           Ice Cream Shop  0.05
4               Playground  0.05


----Etelä-Haaga----
                 venue  freq
0             Bus Stop   0.5
1     Sushi Restaurant   0.1
2          Pizza Place   0.1
3  Sporting Goods Shop   0.1
4                 Café   0.1


----Etelä-Laajasalo----
               venue  freq
0  Convenience Store  0.25
1           Bus Stop  0.25
2               Café  0.25
3              Canal  0.25
4                ATM  0.00


----Etelä-Vuosaari----
           venue  freq
0            ATM  0.25
1  Boat or Ferry  0.25
2           Port  0.25
3       Bus Stop  0.25
4           Pier  0.00


----Etu-Vallila - Alppila----
            venue  freq
0  Sandwich 

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Area']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Area'] = helsinki_grouped['Area']

for ind in np.arange(helsinki_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(helsinki_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aurinkolahti,Grocery Store,Beach,Playground,Gym / Fitness Center,Bridge,Ice Cream Shop,Sri Lankan Restaurant,Resort,Restaurant,Canal
1,Eira - Hernesaari,Park,Scandinavian Restaurant,Ice Cream Shop,Italian Restaurant,Pizza Place,French Restaurant,Boat or Ferry,Bakery,Playground,Modern European Restaurant
2,Etelä-Haaga,Bus Stop,Supermarket,Pizza Place,Sporting Goods Shop,Café,Sushi Restaurant,Diner,Disc Golf,Food & Drink Shop,Food
3,Etelä-Laajasalo,Convenience Store,Bus Stop,Café,Canal,Zoo,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market
4,Etelä-Vuosaari,ATM,Boat or Ferry,Port,Bus Stop,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market


Clustering with K-means clusters

In [72]:
kclusters = 6

helsinki_grouped_clustering = helsinki_grouped.drop('Area', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(helsinki_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 4, 4, 4, 1, 1, 1, 5, 0])

In [73]:
neighbourhoods_venues_sorted.drop(columns=['Cluster Labels'],inplace=True)

neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

helsinki_merged = helsinki_df

helsinki_merged = helsinki_merged.join(neighbourhoods_venues_sorted.set_index('Area'), on='Area')

helsinki_merged.head() 

,Area,newLon,newLat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Pajamäki,24.862298,60.218664,4,Garden,Park,Gym / Fitness Center,Bus Stop,Zoo,Event Space,Food,Flower Shop,Flea Market,Fish Market
1,Kivihaka,24.903746,60.209473,4,Bus Stop,Forest,Recreation Center,Sports Club,Sporting Goods Shop,Electronics Store,Flower Shop,Flea Market,Fish Market,Film Studio
2,Meilahden sairaala-alue,24.910364,60.189822,1,Café,Park,Tram Station,Thai Restaurant,Coffee Shop,College Cafeteria,Pharmacy,Sandwich Place,Scandinavian Restaurant,Scenic Lookout
3,Herttoniemi,25.024531,60.189653,1,Gym / Fitness Center,Harbor / Marina,Garden,Himalayan Restaurant,Mediterranean Restaurant,Pharmacy,Café,Clothing Store,Pier,Grocery Store
4,Ruskeasuo,24.901674,60.202758,1,Plaza,Flea Market,Spa,Sports Club,Pharmacy,Tram Station,Taxi Stand,Himalayan Restaurant,Garden,Forest


Check all the clusters

In [89]:
helsinki_merged.loc[helsinki_merged['Cluster Labels'] == 0, helsinki_merged.columns[[0] + list(range(4, helsinki_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Puotila,Bus Stop,Grocery Store,Playground,Zoo,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
29,Veräjämäki,Bus Stop,Beach,Zoo,Event Space,Forest,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
48,Tuomarinkylä-Torppar,Bus Stop,Bistro,Playground,Zoo,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
57,Itä-Pakila,Bus Stop,Grocery Store,Pizza Place,Historic Site,Playground,Zoo,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Ethiopian Restaurant
66,Metsälä-Etelä-Oulunkylä,Bus Stop,Grocery Store,Playground,Park,Forest,Zoo,Event Space,Food,Flower Shop,Flea Market
71,Vartioharju,Pub,Bus Stop,Zoo,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Film Studio


In [90]:
helsinki_merged.loc[helsinki_merged['Cluster Labels'] == 1, helsinki_merged.columns[[0] + list(range(4, helsinki_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Meilahden sairaala-alue,Café,Park,Tram Station,Thai Restaurant,Coffee Shop,College Cafeteria,Pharmacy,Sandwich Place,Scandinavian Restaurant,Scenic Lookout
3,Herttoniemi,Gym / Fitness Center,Harbor / Marina,Garden,Himalayan Restaurant,Mediterranean Restaurant,Pharmacy,Café,Clothing Store,Pier,Grocery Store
4,Ruskeasuo,Plaza,Flea Market,Spa,Sports Club,Pharmacy,Tram Station,Taxi Stand,Himalayan Restaurant,Garden,Forest
5,Pohjois-Meilahti,Park,Gym / Fitness Center,Bar,Tram Station,Disc Golf,Tennis Court,Scandinavian Restaurant,Bus Stop,Dog Run,Gym Pool
6,Suomenlinna,Café,History Museum,Historic Site,Pier,Harbor / Marina,Sauna / Steam Room,Theater,Bar,Bistro,Tourist Information Center
8,Santahamina,Lounge,Soccer Field,Museum,Fast Food Restaurant,Gym,Cafeteria,Fraternity House,Disc Golf,Discount Store,Food & Drink Shop
10,Pikku Huopalahti,Bar,Tram Station,Playground,Park,Scandinavian Restaurant,Dog Run,Bus Stop,Gym Pool,Supermarket,Plaza
14,Kaivopuisto - Ullanlinna,Scandinavian Restaurant,Park,Pizza Place,Grocery Store,Liquor Store,Ice Cream Shop,Coffee Shop,Cocktail Bar,Sushi Restaurant,French Restaurant
15,Kalasatama,Café,Canal,Art Gallery,Zoo,Bridge,Pub,Bistro,Skate Park,Basketball Court,Park
16,Pohjois-Vuosaari,Other Great Outdoors,Convenience Store,Lake,Pizza Place,Shopping Mall,Bus Stop,Flower Shop,Bar,Supermarket,Hobby Shop


In [91]:
helsinki_merged.loc[helsinki_merged['Cluster Labels'] == 2, helsinki_merged.columns[[0] + list(range(4, helsinki_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,Myllypuro,Flea Market,Mini Golf,Zoo,Ethiopian Restaurant,Forest,Food & Drink Shop,Food,Flower Shop,Fish Market,Film Studio


In [92]:
helsinki_merged.loc[helsinki_merged['Cluster Labels'] == 3, helsinki_merged.columns[[0] + list(range(4, helsinki_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Itäsalmi,Café,Beach,Zoo,Event Space,Forest,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market


In [93]:
helsinki_merged.loc[helsinki_merged['Cluster Labels'] == 4, helsinki_merged.columns[[0] + list(range(4, helsinki_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Pajamäki,Garden,Park,Gym / Fitness Center,Bus Stop,Zoo,Event Space,Food,Flower Shop,Flea Market,Fish Market
1,Kivihaka,Bus Stop,Forest,Recreation Center,Sports Club,Sporting Goods Shop,Electronics Store,Flower Shop,Flea Market,Fish Market,Film Studio
11,Koskela-Helsinki,Bus Stop,Grocery Store,Coffee Shop,Gym Pool,Pharmacy,Hotel,Scenic Lookout,Plaza,Scandinavian Restaurant,Kebab Restaurant
12,Lassila,Bus Stop,Pizza Place,Cafeteria,Train Station,Pharmacy,Skate Park,Karaoke Bar,Bar,Platform,Chinese Restaurant
13,Pohjois-Haaga,Bus Stop,Pizza Place,Platform,Café,Zoo,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market
18,Kulosaari,Garden,Bus Stop,Harbor / Marina,Badminton Court,Zoo,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market
25,Roihuvuori,Bus Stop,Park,Pizza Place,Café,Harbor / Marina,Grocery Store,Zoo,Falafel Restaurant,Food,Flower Shop
28,Viikki,Bus Stop,Recreation Center,Park,Forest,River,Skating Rink,Grocery Store,Stables,Playground,Fish Market
37,Roihupellon teollisuusalue,Bus Stop,Supermarket,Sandwich Place,Himalayan Restaurant,Chinese Restaurant,Grocery Store,Scandinavian Restaurant,Gym / Fitness Center,Liquor Store,Outlet Store
39,Pihlajamäki,Bus Stop,Soccer Field,Flea Market,Pizza Place,Park,Playground,Basketball Court,Grocery Store,Skate Park,Other Great Outdoors


In [94]:
helsinki_merged.loc[helsinki_merged['Cluster Labels'] == 5, helsinki_merged.columns[[0] + list(range(4, helsinki_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Ilmala,Trail,Post Office,Dumpling Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Film Studio,Filipino Restaurant
